# Simulation of a M/M/1 queue

## Efficiency improvement

### Control Variate

In [6]:
include("tally.jl")

stdev (generic function with 1 method)

In [7]:
include("tallystore.jl")

add (generic function with 2 methods)

In [8]:
using SimJulia
using Distributions
using RandomStreams

const SEED = 12345

rand_dist(rng::MRG32k3a, Dist::Distribution) = quantile(Dist, rand(rng))

seeds = [SEED, SEED, SEED, SEED, SEED, SEED]
gen = MRG32k3aGen(seeds)

Seed for next MRG32k3a generator:
[12345,12345,12345,12345,12345,12345]

In [9]:
function ci_normal(n::Int64, mean::Float64, stdev::Float64, α::Float64)
    z = quantile(Normal(), 1-α/2)
    w = z*stdev/sqrt(n)
    # Lower bound
    l = mean - w
    # Upper bound
    u = mean + w
    
    return l, u
end

ci_normal (generic function with 1 method)

In [10]:
function ci_tdist(n::Int64, mean::Float64, stdev::Float64, α::Float64)
    z = quantile(TDist(n-1), 1-α/2)
    w = z*stdev/sqrt(n)
    # Lower bound
    l = mean - w
    # Upper bound
    u = mean + w
    
    return l, u
end

ci_tdist (generic function with 1 method)

In [11]:
function ci_ratio(x, y::TallyStore, α::Float64)
    n = nobs(x.t)
    z = quantile(TDist(n-1), 1-α/2)

    μ1 = average(x.t)
    μ2 = average(y.t)
    # Ratio estimator
    if (μ2 == 0)
        ν = NaN
    else
        ν = μ1/μ2
    end
    
    # Variance of the ratio
    σ = sqrt((variance(x.t) + variance(y.t)*ν*ν - 2*cov(x.obs, y.obs)*ν)/(μ2*μ2))

    w = z*σ/sqrt(n)

    # Lower bound
    l = ν - w
    # Upper bound
    u = ν + w
    
    return ν, l, u
end

ci_ratio (generic function with 1 method)

In [13]:
println("M/M/1 with processes")

# rates
λ = 150.0
μ = 300.0
ρ = λ/μ

# Working periods
periods = 13
# Period length
period_length = 1.0

# occupancy factor
B = [0.8, 1.0, 1.2, 1.4]
pB = [0.25, 0.55, 0.15, 0.05]

# We consider independent days, so there is no warmup to apply.
warmup = 0;

type System
    W
    B
    arrival
    service
    counter
    arrgen
    servgen
    day
    
    calls
    limit
    
    function System(text::String)
        s = new()
        s.W = Tally(text)
        s.B = 1.0
        s.arrival = Exponential(1.0/λ)
        s.service = Exponential(1.0/μ)
        s.arrgen = next_stream(gen)
        s.servgen = next_stream(gen)
        s.day = next_stream(gen)
        
        s.calls = 0
        s.limit = 1.0/180.0
        
        return s;
    end
end

function restart(s::System)
    init(s.W)
    s.calls = 0
    next_substream!(s.arrgen)
    next_substream!(s.servgen)
    next_substream!(s.day)
end

function set_busynessFactor(s::System, B::Float64)
    s.B = B
    s.arrival = Exponential(1.0/(B*λ))
end

M/M/1 with processes


set_busynessFactor (generic function with 1 method)

In [14]:
# Compute the expectation of B
EB = pB'*B

1-element Array{Float64,1}:
 1.0

In [15]:
# Expected arrivals
EA = periods*period_length*λ

1950.0

In [21]:
s = System("Waiting Times")

System(Tally("Waiting Times",0,0.0,Inf,-Inf,0.0,0.0,0.0),1.0,Distributions.Exponential{Float64}(θ=0.006666666666666667),Distributions.Exponential{Float64}(θ=0.0033333333333333335),#undef,Full state of MRG32k3a generator:
Cg = [12345,12345,12345,12345,12345,12345]
Bg = [12345,12345,12345,12345,12345,12345]
Ig = [12345,12345,12345,12345,12345,12345],Full state of MRG32k3a generator:
Cg = [3692455944,1366884236,2968912127,335948734,4161675175,475798818]
Bg = [3692455944,1366884236,2968912127,335948734,4161675175,475798818]
Ig = [3692455944,1366884236,2968912127,335948734,4161675175,475798818],Full state of MRG32k3a generator:
Cg = [1015873554,1310354410,2249465273,994084013,2912484720,3876682925]
Bg = [1015873554,1310354410,2249465273,994084013,2912484720,3876682925]
Ig = [1015873554,1310354410,2249465273,994084013,2912484720,3876682925],0,0.005555555555555556)

In [16]:
# Allow a simulation with a fixed number of clients

function source(env::Environment, s::System, limit::Float64)
    i = 0
    while (true)
        yield(Timeout(env, rand_dist(s.arrgen, s.arrival)))
        if (now(env) > limit) break end
        i += 1
        Process(env, customer, s, i, s.counter)
    end
end

function source_fixed(env::Environment, s::System, counter::Resource, nCusts::Int64)
    for i = 1:nCusts
        yield(Timeout(env, rand_dist(s.arrgen, s.arrival)))
        Process(env, customer, s, i, counter)
    end
end

function customer(env::Environment, s::System, idx::Int, counter::Resource)
    # Record the arrival time in the system
    arrive = now(env)
    yield(Request(counter))
    # The simulation clock now contains the time when the client goes to the server.
    # Record the waiting time
    if (idx > warmup)
        wait = now(env) - arrive
        add(s.W, wait)
        if (wait <= s.limit)
            s.calls += 1
        end
    end
    yield(Timeout(env, rand_dist(s.servgen, s.service)))
    yield(Release(counter))
end

customer (generic function with 1 method)

In [18]:
function gen_day(s::System)
    u = rand(s.day)
    i = 1
    p = pB[1]
    while (p < u)
        i += 1
        p += pB[i]
    end
    s.B = B[i]
    s.arrival = Exponential(1.0/(s.B*λ))
    # println("B: $(s.B), u: $u, i: $i")
end

gen_day (generic function with 1 method)

In [19]:
function onesim(s::System)    
    env = Environment()
    s.counter = Resource(env, 1)

    # We consider a system over 24 hours
    Process(env, source, s, periods*period_length)
    run(env)
end

onesim (generic function with 1 method)

In [22]:
# We genereate the day-dependent quantities
gen_day(s)
    
onesim(s)

In [23]:
average(s.W)

0.0033006779213345626

In [24]:
nobs(s.W)

1972

In [25]:
s.calls

1558

In [26]:
# Service levels
SL = s.calls/nobs(s.W)

0.7900608519269777

In [27]:
meanWaits = TallyStore("Average waiting times")

TallyStore(Float64[],Tally("Average waiting times",0,0.0,Inf,-Inf,0.0,0.0,0.0))

In [28]:
totWaits = TallyStore("Cumulative waiting times")

TallyStore(Float64[],Tally("Cumulative waiting times",0,0.0,Inf,-Inf,0.0,0.0,0.0))

In [29]:
nObs = TallyStore("Number of observations")
nCalls = TallyStore("Number of calls answered in less than $(s.limit*3600) seconds")
nOver = TallyStore("Number of calls answered in more than $(s.limit*3600) seconds")

TallyStore(Float64[],Tally("Number of calls answered in more than 20.0 seconds",0,0.0,Inf,-Inf,0.0,0.0,0.0))

In [30]:
nSim = 100

100

In [31]:
for n=1:nSim
    restart(s)
    gen_day(s)
    onesim(s)
    w = average(s.W)
    N = nobs(s.W)
    add(meanWaits, w)
    add(totWaits, sum(s.W))
    add(nObs, Float64(N))
    add(nCalls, Float64(s.calls))
    add(nOver, Float64(N-s.calls))
    println("Sim $n. Average waiting time: $w. Number of observations: $N")
end

Sim 1. Average waiting time: 0.003720571871425421. Number of observations: 1941
Sim 2. Average waiting time: 0.0033023487299231784. Number of observations: 1946
Sim 3. Average waiting time: 0.003308842792981638. Number of observations: 1985
Sim 4. Average waiting time: 0.0020712515586998885. Number of observations: 1599
Sim 5. Average waiting time: 0.00318239428812974. Number of observations: 1969
Sim 6. Average waiting time: 0.0022165349423393715. Number of observations: 1565
Sim 7. Average waiting time: 0.0031093091894904513. Number of observations: 1911
Sim 8. Average waiting time: 0.0035067456703061703. Number of observations: 1913
Sim 9. Average waiting time: 0.004361536640892026. Number of observations: 2332
Sim 10. Average waiting time: 0.002788153049901509. Number of observations: 1543
Sim 11. Average waiting time: 0.0033044579571697465. Number of observations: 2009
Sim 12. Average waiting time: 0.005351663428118502. Number of observations: 2354
Sim 13. Average waiting time: 0.

In [32]:
average(nObs.t)

1956.9199999999994

In [33]:
average(nCalls.t)

1498.4399999999998

In [34]:
variance(nCalls.t)

11962.854949494951

In [35]:
variance(nCalls.t)/nSim

119.62854949494951

In [36]:
# Indirect estimator
EA-average(nOver.t)

1491.52

In [37]:
variance(nOver.t)

48599.908686868665

In [38]:
covariance = cov(nCalls.obs, nObs.obs)

27481.611313131314

In [39]:
# Estimator with control variate
average(nCalls.t)-covariance*(average(nObs.t)-EA)/variance(nObs.t)

1496.3638845357636

In [40]:
# Construct the new estimator
x_vc = Tally("Control Variate estimator")

for i = 1:nSim
    add(x_vc, nCalls.obs[i]-covariance*(nObs.obs[i]-EA)/variance(nObs.t))
end

In [41]:
average(x_vc)

1496.363884535764

In [42]:
variance(x_vc)

3717.9130088411243

### Stratification

In [64]:
# We first define the number of observations per strate
nB = length(B)
nDays = Array(Int64, nB)
for i = 1:nB
    nDays[i] = round(nSim*pB[i])
end

In [65]:
nDays

4-element Array{Int64,1}:
  50
 110
  30
  10

Note: this allocation could be improved by ensuring that each strate has enough observations and that the sum is equal to the number of simulations previously set.

In [66]:
servedCalls = Array(Tally, nB)

for i=1:nB
    servedCalls[i] = Tally("Served calls in time for strate %i")
end

In [67]:
for i = 1:nB
    set_busynessFactor(s, B[i])
    for n=1:nDays[i]
        restart(s)
        onesim(s)
        add(servedCalls[i], Float64(s.calls))
    end
end

In [68]:
# Strate details

for i = 1:nB
    println("Strate $i. Mean: ", average(servedCalls[i]),
            " Variance: ", variance(servedCalls[i]))
end

Strate 1. Mean: 1335.3 Variance: 1677.6428571428562
Strate 2. Mean: 1523.6272727272733 Variance: 2157.5203502919044
Strate 3. Mean: 1613.7666666666667 Variance: 2814.8057471264337
Strate 4. Mean: 1552.5 Variance: 2472.277777777778


In [72]:
mean = 0.0
for i = 1:nB
    mean += pB[i]*average(servedCalls[i])
end

In [73]:
mean

1491.5100000000004

In [74]:
σ2 = 0.0
for i = 1:nB
    σ2 += pB[i]*pB[i]*variance(servedCalls[i])/nobs(servedCalls[i])
end
σ2 *= nSim

2151.881657904116

In [75]:
total = 0
nSim = 200
for i=1:nB
    nDays[i] = round(pB[i]*stdev(servedCalls[i]))
    total += nDays[i]
end
for i = 1:nB
    nDays[i] = round(nDays[i]*nSim/total)
end

In [76]:
nDays

4-element Array{Int64,1}:
  43
 113
  35
   9

In [77]:
for i=1:nB
    servedCalls[i] = Tally("Served calls in time for strate %i")
end

for i = 1:nB
    set_busynessFactor(s, B[i])
    for n=1:nDays[i]
        restart(s)
        onesim(s)
        add(servedCalls[i], Float64(s.calls))
    end
end

In [78]:
# Strate details

for i = 1:nB
    println("Strate $i. Mean: ", average(servedCalls[i]),
            " Variance: ", variance(servedCalls[i]))
end

Strate 1. Mean: 1331.9302325581398 Variance: 1472.3045404208187
Strate 2. Mean: 1522.8761061946902 Variance: 1675.7880847029064
Strate 3. Mean: 1602.7428571428572 Variance: 2005.9025210083983
Strate 4. Mean: 1579.0 Variance: 3685.2499999999995


In [79]:
σ2 = 0.0
for i = 1:nB
    σ2 += pB[i]*pB[i]*variance(servedCalls[i])/nobs(servedCalls[i])
end
σ2 *= nSim

1787.8473443018017